In [1]:
import nltk

In [2]:
nltk.download("twitter_samples")

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
from nltk.corpus import twitter_samples
nltk.download("twitter_samples")

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [4]:
import nltk                                  # Python library for NLP
from nltk.corpus import twitter_samples      # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt              # visualization library
import numpy as np                           # library for scientific computing and matrix operations

In [5]:
# downloads sample twitter dataset.
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [6]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import re
import string
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [10]:
pos=twitter_samples.strings("positive_tweets.json")
neg=twitter_samples.strings("negative_tweets.json")
tweets=pos+neg
labels=np.append(np.ones((len(pos))),np.zeros((len(neg))))


In [11]:
len(tweets)

10000

In [12]:
from sklearn.model_selection import train_test_split
train_tweets,test_tweets,train_lab,test_lab=train_test_split(tweets,labels,train_size=0.8,random_state=42)

In [13]:
len(test_tweets)

2000

In [14]:
len(test_lab[test_lab==1])

1012

In [15]:
train_lab=train_lab.reshape(8000,1)
test_lab=test_lab.reshape(2000,1)

In [16]:
def process_tweet(tweet):
    stemmer=PorterStemmer()
    eng_stop=stopwords.words("english")
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer=TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
    tweet_tokens=tokenizer.tokenize(tweet)
    tweets_clean=[]
    for word in tweet_tokens:
        if word not in eng_stop and word not in string.punctuation :
            stem=stemmer.stem(word)
            tweets_clean.append(stem)
            
    return tweets_clean

In [17]:
def freqs(tweets,ys):
    yslist = np.squeeze(ys).tolist()
    freq={}
    for y,tweet in zip(yslist,tweets):
        for word in process_tweet(tweet):
            p=(word,y)
            if p in freq:
                freq[p]+=1
            else:
                freq[p]=1
    return freq

In [18]:
frequency=freqs(tweets,labels)

In [19]:
len(frequency)

13067

In [20]:
keys = ['happi', 'merri', 'nice', 'good', 'bad', 'sad', 'mad', 'best', 'pretti',
        '❤', ':)', ':(', '😒', '😬', '😄', '😍', '♛',
        'song', 'idea', 'power', 'play', 'magnific']
data=[]
for word in keys:
    po=0
    ne=0
    if (word,1) in frequency:
        po=frequency[(word,1)]
    if (word,0) in frequency:
        ne=frequency[(word,0)]
    data.append([word,po,ne])

data

[['happi', 211, 25],
 ['merri', 1, 0],
 ['nice', 98, 19],
 ['good', 238, 101],
 ['bad', 18, 73],
 ['sad', 5, 123],
 ['mad', 4, 11],
 ['best', 65, 22],
 ['pretti', 20, 15],
 ['❤', 29, 21],
 [':)', 3568, 2],
 [':(', 1, 4571],
 ['😒', 1, 3],
 ['😬', 0, 2],
 ['😄', 5, 1],
 ['😍', 2, 1],
 ['♛', 0, 210],
 ['song', 22, 27],
 ['idea', 26, 10],
 ['power', 7, 6],
 ['play', 46, 48],
 ['magnific', 2, 0]]

In [21]:
freq_train=freqs(train_tweets,train_lab)

In [22]:
len(freq_train)

11372

In [23]:
from sklearn.linear_model import LogisticRegression
Mod=LogisticRegression()


In [24]:
def features(tweet,freq):
    v=process_tweet(tweet)
    x=np.zeros((1,3))
    x[0,0]=1#bias term
    for word in v:
        if (word,1) in freq:
            x[0,1]+=freq[(word,1)]
        if (word,0) in freq:
            x[0,2]+=freq[(word,0)]
            
    return x
            

In [25]:
x_train=np.zeros((len(train_tweets),3))
for i in range(len(train_tweets)):
    x_train[i,:]=features(train_tweets[i],freq_train)

In [26]:
y_train=train_lab
y_train=y_train.reshape(8000,)

In [27]:
Mod.fit(x_train,y_train)

LogisticRegression()

In [28]:
y=Mod.predict(x_train)

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [30]:
print(accuracy_score(y_train,y))

0.990125


In [31]:
print(confusion_matrix(y_train,y))

[[3948   64]
 [  15 3973]]


In [32]:
freq_test=freqs(test_tweets,test_lab)

In [33]:
x_test=np.zeros((len(test_tweets),3))
for i in range(len(test_tweets)):
    x_test[i,:]=features(test_tweets[i],freq_test)

In [34]:
y_test=test_lab
y_test.shape
y_test=y_test.reshape(2000,)

In [35]:
y=Mod.predict(x_test)

In [36]:
print(accuracy_score(y_test,y))
print(confusion_matrix(y_test,y))
print(classification_report(y_test,y))

0.991
[[ 970   18]
 [   0 1012]]
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99       988
         1.0       0.98      1.00      0.99      1012

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [37]:
#NAIVE BAYES

In [38]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
model=MultinomialNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)


In [40]:
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9945
[[ 982    6]
 [   5 1007]]
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       988
         1.0       0.99      1.00      0.99      1012

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [64]:
tweet="The movie is bad"
#ys=np.ones((1,))
#f=freqs(tweet,ys)
x_st=np.zeros((1,3))
x_st[0,:]=features(tweet,freq_train)
y=model.predict(x_st)

In [65]:
y

array([0.])